In [2]:
import numpy as np
from time import time

### 1 Why not use loop
---

#### 1.1 Arrays are mutable

In [3]:
x_array = np.arange(5)
x_array

array([0, 1, 2, 3, 4])

In [4]:
x_array[0] = 99
x_array

array([99,  1,  2,  3,  4])

In [5]:
x_array[3:6] = [99,99] # this is permissable since the array returned by the slice is of the same shape as the assignment array
x_array

array([99,  1,  2, 99, 99])

#### 1.2 Loops are slow

In [6]:
bigmatrix = np.random.random((1000,2000))
bigmatrix.size

2000000

In [ ]:
# A loop based computation of the sum of the max of all rows

def sum_of_max_loop(bigmatrix):
    
    m,n = bigmatrix.shape
    
    maxValues = np.zeros(shape=(m,))
    
    for i in range(m):
        for j in range(n):
            if(bigmatrix[i,j] > maxValues[i]):
                maxValues[i] = bigmatrix[i,j]